In [ ]:
# Imports
import time
import copy
import numpy as np
import pandas as pd
from enum import Enum

import panel as pn
import param

from pymepix.channel.channel_types import ChannelDataType, Commands

from online_data.timepix_monitor.tpx_acquisition import initialize_timepix_monitor 
from online_data.timepix_monitor.tpx_acquisition import unregister_all as tpxmon_unregister_all
from online_data.timepix_monitor.tpx_acquisition import callback as tpxmon_callback

from online_data.scan_monitor.scan_acquisition import callback as scanmon_callback
from online_data.scan_monitor.scan_acquisition import initialize_scan_monitor
from online_data.scan_monitor.scan_acquisition import unregister_all as scanmon_unregister_all

import holoviews as hv
from holoviews import opts
from holoviews import streams
from holoviews.streams import Pipe
from bokeh.plotting import show
from holoviews import opts

hv.extension('bokeh')
pn.extension('texteditor')
#pn.extension()

In [ ]:
initialize_timepix_monitor()
initialize_scan_monitor()

## Globals ##

In [ ]:
TOF_BINS_NUMBER = 1000
TOF_RANGE = 10.0
TOF_YLOG = False

bins_VMI = (range(0, 256), range(0, 256))
H_VMI = np.zeros((255, 255), dtype=float)

bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
x_axe = bins_series[:-1]

class States(Enum):
    ON = 1
    SCANNING = 2
    
scan_info = {}
scan_step = 0
    
State = States.ON


scan_data = []
scan_table = pd.DataFrame()


full_range_roi_init_data = {
    'full_range':{
        'tof_roi': (0,TOF_RANGE),
        'spatial_roi': ((0,0), (255,255)),
        'roi_tof_histo': np.zeros((len(bins_series)-1), dtype=float),
        'roi_H_VMI': np.zeros((255, 255), dtype=float),
        'roi_color': '#99ef78',
        'n_tiggers': 0
    }
}
    
scan_data.append(full_range_roi_init_data)  #always at least one record data


pipe_roi_scan = Pipe(data=pd.DataFrame())
pipe_image = Pipe(data=[])
pipe_tof = Pipe(data=[])

pipe_roi_scan.send(scan_table)
pipe_image.send(H_VMI)
pipe_tof.send(np.zeros((len(bins_series)-1),) )

### Image&Spectrum panel  ###

In [ ]:
def on_roi_select_change(Event):
    pass    

def reset_button_callback(Event):
    global scan_data, scan_table
    
    newRecord = copy.deepcopy(scan_data[-1])
    for roi_name, roi_record in newRecord.items():
        roi_record['roi_tof_histo'] = np.zeros((len(bins_series)-1), dtype=float)
        roi_record['roi_H_VMI'] =  np.zeros((255, 255), dtype=float)
        roi_record['n_tiggers'] = 0
    scan_data = [newRecord]
    scan_table = pd.DataFrame()
    
roi_select = pn.widgets.MultiChoice(
    name='ToF/mass ROI', value=list(scan_data[-1].keys()),
    options=list(scan_data[-1].keys()),)

reset_button = pn.widgets.Button(name='Reset data', button_type='primary')
reset_button.on_click(reset_button_callback)

roi_select.param.watch(on_roi_select_change, 'value')

### mass calibration panel ###

In [ ]:
mass_calibration_data = None # otherwise it is tuple(A, t0), m=A(t-t0)

df_widget = pn.widgets.DataFrame(pd.DataFrame(), height=800, frozen_columns=1, autosize_mode='none')

time_point1 = pn.widgets.FloatInput(name='Time 1', value=0., start=0, end=TOF_RANGE)
time_point2 = pn.widgets.FloatInput(name='Time 2', value=0., start=0, end=TOF_RANGE)
mass_point1 = pn.widgets.FloatInput(name='Mass 1', value=0.,)
mass_point2 = pn.widgets.FloatInput(name='Mass 2', value=0.,)

def calculate_mass_from_tof(time_series):
    if mass_calibration_data != None:
        return mass_calibration_data[0]*pow((time_series - mass_calibration_data[1]),2)
    return time_series

def mass_calibrate_callback(Event):
    global mass_calibration_data, bins_series, x_axe
    
    mass1_sqrt = pow(mass_point1.value,0.5)
    mass2_sqrt = pow(mass_point2.value,0.5)
    
    t0 = (mass2_sqrt*time_point1.value - mass1_sqrt*time_point2.value)/(mass2_sqrt-mass1_sqrt)
    A = mass_point2.value/pow((time_point2.value - t0),2)
    
    mass_calibration_data = (A, t0)
    
    mass_calib_string.value = f'{A}  {t0}'
    
    bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
    bins_series = calculate_mass_from_tof(bins_series)
    
    min_mass_index = np.argmin(bins_series)
    if min_mass_index > 0:
        bins_series[:min_mass_index] = -bins_series[:min_mass_index]
   
    x_axe = bins_series[:-1]

def reset_calibration_callback(Event):
    global mass_calibration_data, bins_series, x_axe
    mass_calibration_data = None
    bins_series = np.linspace(0, TOF_RANGE, TOF_BINS_NUMBER)
    x_axe = bins_series[:-1]
    mass_calib_string.value = 'None'

mass_calibrate_button = pn.widgets.Button(name='Calibrate ToF spectra', button_type='primary')
mass_calibrate_button.on_click(mass_calibrate_callback)

reset_calibration_button = pn.widgets.Button(name='Reset mass calibration', button_type='primary')
reset_calibration_button.on_click(reset_calibration_callback)

mass_calib_string = pn.widgets.StaticText(name='Calibration const.', value='None')



### ROI management panel ###

In [ ]:
m1_point = pn.widgets.FloatInput(name='m1', value=0., start=0)
m2_point = pn.widgets.FloatInput(name='m2', value=0., start=0)

x1_point = pn.widgets.IntInput(name='X1', value=0,   start=0, end=255)
x2_point = pn.widgets.IntInput(name='X2', value=255, start=0, end=255)
y1_point = pn.widgets.IntInput(name='Y1', value=0,   start=0, end=255)
y2_point = pn.widgets.IntInput(name='Y2', value=255, start=0, end=255)

text_input_roi_name = pn.widgets.TextInput(name='ROI name', placeholder='Enter the name of ROI here...')

colorpicker = pn.widgets.ColorPicker(name='ROI color', value='#99ef78')

def add_roi_callback(Event):
    
    if text_input_roi_name.value == '':
        return
    
    new_roi= {
            'tof_roi': (m1_point.value, m2_point.value),
            'spatial_roi': ((x1_point.value, y1_point.value), (x2_point.value,y2_point.value)),
            'roi_tof_histo': np.zeros((len(bins_series)-1), dtype=float),
            'roi_H_VMI': np.zeros((255, 255), dtype=float),
            'roi_color': colorpicker.value,
            'n_tiggers': 0
    }
    
    scan_data[-1][text_input_roi_name.value] = new_roi
    
    if not (text_input_roi_name.value in roi_select.options):
        options_list = roi_select.options.copy()
        options_list.append(text_input_roi_name.value)
        roi_select.options = []
        roi_select.options = options_list
        roi_select.value=[text_input_roi_name.value]
    

def remove_roi_by_name_callback(Event):
    if text_input_roi_name.value == 'full_range':
        return 
    
    del scan_data[-1][text_input_roi_name.value]
    
    value_list = roi_select.value.copy()
    if text_input_roi_name.value in value_list:
        value_list.remove(text_input_roi_name.value)
        roi_select.value = []
        #roi_select.values = []
    
    if text_input_roi_name.value in roi_select.options:
        options_list = roi_select.options.copy()
        options_list.remove(text_input_roi_name.value)
        roi_select.options = []
        roi_select.options = options_list
        #roi_select.values = []
    
    if roi_select.value == [] and value_list != []:
        roi_select.value = value_list
  


add_roi_button = pn.widgets.Button(name='Add ROI', button_type='primary')
add_roi_button.on_click(add_roi_callback)

remove_roi_by_name_button = pn.widgets.Button(name='Remove ROI by name', button_type='primary')
remove_roi_by_name_button.on_click(remove_roi_by_name_callback)




### New panel ###

## Main callback routine where the data processing takes place ##

In [ ]:
@tpxmon_callback
def main_loop(in_data):
    
    #global tof_accu
    global x_axe
    global scan_data
    global scan_table
    global scan_step
    global roi_select
    
    if not in_data['type'] in [ChannelDataType.TOF.value, ChannelDataType.COMMAND.value]:
        return

    if in_data['type'] == ChannelDataType.TOF.value:
        
        last_record_data = scan_data[-1]
        
        tof_data = in_data['data']
        mass_series = calculate_mass_from_tof(np.copy(tof_data['tof']*1e6)) #timeseries is in microseconds, mass calibration
        
        
        last_record_data['full_range']['roi_H_VMI'] += np.histogram2d(tof_data['x'], tof_data['y'], bins=bins_VMI)[0]
        for roi_key, roi_val in last_record_data.items():
            roi_indxs = np.logical_and(mass_series>roi_val['tof_roi'][0], mass_series<roi_val['tof_roi'][1])
            roi_val['roi_H_VMI'] += np.histogram2d(tof_data['x'][roi_indxs], tof_data['y'][roi_indxs], bins=bins_VMI)[0]


        if roi_select.value == ['full_range']:
            roi_H_VMI = last_record_data['full_range']['roi_H_VMI']
        else:
            roi_H_VMI = np.zeros((255, 255), dtype=float)
            for roi_key, roi_val in last_record_data.items():
                if roi_key == 'full_range':
                    continue
                if roi_key in roi_select.value:
                    roi_H_VMI += roi_val['roi_H_VMI']
        pipe_image.send(roi_H_VMI)

        last_record_data['full_range']['roi_tof_histo'] += np.histogram(mass_series, bins=bins_series)[0]/len(np.unique(tof_data['nr']))
        series_data = last_record_data['full_range']['roi_tof_histo']
        pipe_tof.send(series_data/max(series_data.max(),1))
        
        last_record_data['full_range']['n_tiggers'] += len(np.unique(tof_data['nr'])) # triggers number stored only here
        return


    if in_data['type'] == ChannelDataType.COMMAND.value:  
        if in_data['data'] == Commands.START_RECORD.value:
            # init new record data with all ROIs
            newRecord = copy.deepcopy(scan_data[-1])
            for roi_name, roi_record in newRecord.items():
                roi_record['roi_tof_histo'] = np.zeros((len(bins_series)-1), dtype=float)
                roi_record['roi_H_VMI'] =  np.zeros((255, 255), dtype=float)
                roi_record['n_tiggers'] = 0
            scan_data.append(newRecord)

        elif in_data['data'] == Commands.STOP_RECORD.value:
            # fill data frame with data from ROIs            
            vals = []
            roi_names = []

            for roi_name, roi_record in scan_data[-1].items():
                if roi_name == 'full_range':
                    continue
                roi_names.append(roi_name)

                pix_roi = roi_record['spatial_roi']
                val = roi_record['roi_H_VMI'][pix_roi[0][0]:pix_roi[1][0], pix_roi[0][1]:pix_roi[1][1]]
                if scan_data[-1]['full_range']['n_tiggers'] > 0:
                    val = np.sum(val)/scan_data[-1]['full_range']['n_tiggers']
                else:
                    val = 0.0
                vals.append(val)

            if scan_table.empty:
                scan_table = pd.DataFrame(columns=['scan_step'] + roi_names)

            scan_table.loc[scan_step] = [scan_step] + vals

            #sync problems, record end could be recieved before the stop record command from camera
            if State == States.SCANNING or scan_step <= scan_info['scan_intervals']:
                scan_table['scan_step'] = scan_info['scan_space'][0:scan_step+1]
                
            scan_step += 1

            process_scan_roi_data(scan_table)                
                
                    
            pipe_roi_scan.send(scan_table)
        return

In [ ]:
def process_scan_roi_data(in_table):    
    global df_widget
    
    x_centroid = []
    
    for i in range(1,len(scan_table.keys())):
        integral_val = sum(scan_table.iloc[:, i])
        if integral_val > 0:
            x_centroid.append(sum(scan_table.iloc[:, 0]*scan_table.iloc[:, i])/integral_val)
        else:
            x_centroid.append(None)

    df_widget.value = pd.DataFrame({'ROI': in_table.keys()[1:], f'{get_movable_label()}_centroided':\
                                    x_centroid})
 

In [ ]:
# retriving scan information from sardana
@scanmon_callback
def on_scandata_received(in_data): 
    #print(in_data)
    global State
    global scan_info
    global scan_table
    global scan_step
    
    if in_data['type'] == 'data_desc' and State == States.ON:
        State = States.SCANNING # start of scanning
        scan_info['movable_label'] = in_data['data']['column_desc'][1]['label']
        scan_info['min_value'] = in_data['data']['column_desc'][1]['min_value']
        scan_info['max_value'] = in_data['data']['column_desc'][1]['max_value']
        scan_info['scan_intervals'] = in_data['data']['total_scan_intervals']
        scan_info['scan_space'] = np.linspace(scan_info['min_value'],scan_info['max_value'],\
                                              scan_info['scan_intervals']+1)
        scan_info['serial_number'] = in_data['data']['serialno']
        scan_table = pd.DataFrame()
        scan_step = 0
        
        
        
    elif in_data['type'] == 'record_end':
        State = States.ON

## Panel GUI, server start ##

In [ ]:
def get_movable_label():
    global scan_info
    xlabel = 'scan step'
    if 'movable_label' in scan_info.keys():
        xlabel = scan_info['movable_label']
    return xlabel

def make_roi_scan_plot(data):
    global roi_select  
    global State
    global scan_info

    if roi_select.value != [] and data.empty != True:

        roi_names = [i for i in roi_select.value]

        xmax = data['scan_step'].max()
        xmin = data['scan_step'].min()
        ymax = max(data[roi_names].max())
        ymin = min(data[roi_names].min())

        if xmax==xmin:
            xlimits = (xmax-1,xmax+1,)
        else:
            xrange = xmax-xmin
            xlimits = (xmin - xrange*0.1,xmax+xrange*0.1,)

        if ymax==ymin:
            ylimits = (ymax-1,ymax+1,)
        else:
            yrange = ymax-ymin
            ylimits = (ymin - yrange*0.1,ymax+yrange*0.1,)

        colors = {}
        for roi_name, roi in scan_data[-1].items():
            colors[roi_name]=roi['roi_color']
        
        xlabel = get_movable_label()
        
        if 'serial_number' in scan_info.keys():
            figure_title = f"Scan: {scan_info['serial_number']}"
        else:
            figure_title = ''
       
        curves=[hv.Curve((data['scan_step'], data[roi_name])).opts(xlabel=xlabel, \
                            ylabel='Amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'],\
                            color=colors[roi_name], xlim=xlimits, ylim=ylimits, axiswise=True, \
                            framewise=True,)*hv.Scatter((data['scan_step'],\
                            data[roi_name])).opts(xlabel='scan num.',\
                            ylabel='Amplitude, [au]', width=600, height=450, show_grid=True,\
                            tools=['hover'], color=colors[roi_name], xlim=xlimits, ylim=ylimits, axiswise=True,\
                            framewise=True, size=10) for roi_name in roi_select.value if roi_name != 'full_range' ]

               

        return hv.Overlay(curves).opts( title=figure_title, axiswise=True, framewise=True,) #.redim.range(Sample=(0,10))
    else:
        hv_curve = hv.Curve([]).opts(xlabel='scan num.', ylabel='Amplitude, [au]', width=600, height=450, show_grid=True, tools=['hover'],  axiswise=True, framewise=True,)
        return hv.Overlay([hv_curve , hv_curve ],).opts(axiswise=True, framewise=True,)

    
    
def make_image_plot(data):
    
    global roi_select, scan_data
    #print('got image data')
    #print(data)
    rois_coord = [scan_data[-1][roi_name]['spatial_roi'] for roi_name in roi_select.value]
    rois_colors = [scan_data[-1][roi_name]['roi_color'] for roi_name in roi_select.value]

    rois_props = list(zip(rois_coord, rois_colors))

    image_plot = hv.Image((range(256), range(256), data.T), bounds=[bins_VMI[0][0], bins_VMI[1][0], bins_VMI[0][-1], \
                                        bins_VMI[1][-1]],).opts(xlabel='x', ylabel='y', axiswise=True, \
                                        logz=True, clim=(1e-3, None), cmap="jet", frame_height=500, \
                                        frame_width=500)
    
    bounds = [hv.Bounds((roi[0][0][0], roi[0][0][1], roi[0][1][0], roi[0][1][1])).opts(opts.Bounds(color=roi[1],\
                            axiswise=True, line_width=2)) for roi in rois_props ]
    
    return (image_plot * hv.Overlay(bounds).opts(axiswise=True)).opts(axiswise=True)

def make_tof_histo_plot(data):
    global roi_select
    
    last_roi_data = scan_data[-1]
 
    selected_rois = [(last_roi_data[i]['tof_roi'], last_roi_data[i]['roi_color']) for i in roi_select.value if i != 'full_range']
    
    if mass_calibration_data == None:
        xlabel='ToF (µs)'
    else:
        xlabel='Mass (amu)'
        
    ylabel='Normalized amplitude'
    
    tof_hist_plot_log = hv.Curve((x_axe, data)).opts(xlabel=xlabel, ylabel=ylabel, axiswise=True, height=400,\
                                     width=1200, show_grid=True, tools=['hover'], ylim=(1e-7,1),xlim=(x_axe[0],\
                                     x_axe[-1]), logy=TOF_YLOG)
    
    
    #rng = hv.streams.RangeY(source=p)
    
    mass_rois = [hv.Rectangles((i[0], 1e-7, i[1], 1)).opts(alpha=0.3, color=c) for i, c in selected_rois]
    #mass_rois = [hv.Rectangles((i[0], rng[0], i[1], rng[1])).opts(alpha=0.3, color=c) for i, c in selected_rois]
    return tof_hist_plot_log * hv.Overlay(mass_rois)


main_panel = pn.Column(reset_button)

mass_calibrate_panel = pn.Row(pn.Column(time_point1, mass_point1, time_point2, mass_point2,\
                                mass_calibrate_button, reset_calibration_button, mass_calib_string),)

roi_panel = pn.Column(m1_point, m2_point, x1_point, y1_point, x2_point,\
                            y2_point,text_input_roi_name, colorpicker, add_roi_button,\
                            remove_roi_by_name_button)

settings_tabs = pn.Tabs(('Main', main_panel),
        ('Mass calibration', mass_calibrate_panel),
        ('ROI settings',roi_panel))

viz_panel = pn.Tabs(('Plots', pn.Column(roi_select, pn.Row(hv.DynamicMap(make_image_plot, streams=[pipe_image]),\
                                hv.DynamicMap(make_roi_scan_plot, streams=[pipe_roi_scan])),
                                   hv.DynamicMap(make_tof_histo_plot,  streams=[pipe_tof]))),
             ('Tables', df_widget))


page = pn.Row(settings_tabs, viz_panel)


pn.serve(page)


#         , port=SERVE_PORT)